<a href="https://colab.research.google.com/github/MpRonald/Machine-Learning/blob/main/SVM_Semi_Sup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.semi_supervised import SelfTrainingClassifier
import pickle as pk

import warnings
warnings.filterwarnings('ignore')

### Supervioned Model

In [2]:
path = 'https://raw.githubusercontent.com/MpRonald/datasets/main/qualidade_leite.csv'
df = pd.read_csv(path)
df.head()

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
2,8.5,70,1,1,1,1,246,NaN
3,9.5,34,1,1,0,1,255,NaN
4,6.6,37,0,0,0,0,255,média


In [3]:
df.shape

(1059, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pH           1059 non-null   float64
 1   Temperatura  1059 non-null   int64  
 2   Sabor        1059 non-null   int64  
 3   Odor         1059 non-null   int64  
 4   Gordura      1059 non-null   int64  
 5   Turbidez     1059 non-null   int64  
 6   Cor          1059 non-null   int64  
 7   Qualidade    424 non-null    object 
dtypes: float64(1), int64(6), object(1)
memory usage: 66.3+ KB


In [5]:
df.Qualidade.value_counts(normalize=True, dropna=False)

NaN      0.599622
baixa    0.173749
média    0.140699
alta     0.085930
Name: Qualidade, dtype: float64

In [6]:
df.Qualidade.isnull().sum()

635

In [7]:
df.describe()

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
count,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000
mean,6.630123,44.226629,0.546742,0.432483,0.671388,0.491029,251.840415
std,1.399679,10.098364,0.498046,0.495655,0.469930,0.500156,4.307424
min,3.000000,34.000000,0.000000,0.000000,0.000000,0.000000,240.000000
25%,6.500000,38.000000,0.000000,0.000000,0.000000,0.000000,250.000000
50%,6.700000,41.000000,1.000000,0.000000,1.000000,0.000000,255.000000
75%,6.800000,45.000000,1.000000,1.000000,1.000000,1.000000,255.000000
max,9.500000,90.000000,1.000000,1.000000,1.000000,1.000000,255.000000


In [8]:
df.loc[df.Qualidade == 'alta']

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
29,6.6,37,1,0,1,0,255,alta
32,6.8,42,1,1,1,1,255,alta
35,6.8,45,0,1,1,1,255,alta
...,...,...,...,...,...,...,...,...
1001,6.8,45,0,1,1,1,255,alta
1013,6.8,45,0,1,1,1,255,alta
1028,6.6,37,1,0,1,0,255,alta
1043,6.8,40,1,1,1,1,255,alta


In [9]:
df.loc[df.Qualidade == 'alta'].describe()

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
count,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,6.708791,41.076923,0.659341,0.747253,0.989011,0.571429,252.153846
std,0.106090,3.777803,0.476557,0.436995,0.104828,0.497613,3.943836
min,6.500000,35.000000,0.000000,0.000000,0.000000,0.000000,245.000000
25%,6.600000,38.000000,0.000000,0.500000,1.000000,0.000000,250.000000
50%,6.800000,41.000000,1.000000,1.000000,1.000000,1.000000,255.000000
75%,6.800000,45.000000,1.000000,1.000000,1.000000,1.000000,255.000000
max,6.800000,45.000000,1.000000,1.000000,1.000000,1.000000,255.000000


In [10]:
df.loc[df.Qualidade == 'média']

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
4,6.6,37,0,0,0,0,255,média
9,6.7,45,1,1,0,0,247,média
21,6.5,36,0,0,1,0,255,média
24,6.8,35,1,0,1,0,246,média
27,6.6,34,0,0,0,1,240,média
...,...,...,...,...,...,...,...,...
1023,6.5,35,1,0,1,0,246,média
1026,6.8,34,0,0,0,1,240,média
1041,6.6,40,1,0,0,0,255,média
1049,6.5,37,0,0,0,0,255,média


In [11]:
df.loc[df.Qualidade == 'média'].describe()

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
count,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000
mean,6.632215,39.013423,0.402685,0.114094,0.362416,0.140940,250.449664
std,0.134701,3.385156,0.492092,0.318998,0.482319,0.349133,5.390728
min,6.500000,34.000000,0.000000,0.000000,0.000000,0.000000,240.000000
25%,6.500000,37.000000,0.000000,0.000000,0.000000,0.000000,246.000000
50%,6.600000,38.000000,0.000000,0.000000,0.000000,0.000000,255.000000
75%,6.800000,41.000000,1.000000,0.000000,1.000000,0.000000,255.000000
max,6.800000,45.000000,1.000000,1.000000,1.000000,1.000000,255.000000


In [12]:
df.loc[df.Qualidade == 'baixa'].describe()

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
count,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000
mean,6.658152,49.445652,0.603261,0.467391,0.744565,0.739130,252.668478
std,2.164679,12.964739,0.490556,0.500297,0.437295,0.440307,3.212660
min,3.000000,34.000000,0.000000,0.000000,0.000000,0.000000,245.000000
25%,4.700000,40.000000,0.000000,0.000000,0.000000,0.000000,250.000000
50%,6.750000,45.000000,1.000000,0.000000,1.000000,1.000000,255.000000
75%,8.600000,55.000000,1.000000,1.000000,1.000000,1.000000,255.000000
max,9.500000,90.000000,1.000000,1.000000,1.000000,1.000000,255.000000


In [13]:
df.loc[(df['pH'] == 9.5)]

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
3,9.5,34,1,1,0,1,255,NaN
48,9.5,34,1,1,0,1,255,baixa
94,9.5,34,1,1,0,1,255,NaN
96,9.5,34,1,1,0,1,255,NaN
150,9.5,34,1,1,0,1,255,baixa
152,9.5,34,1,1,0,1,255,NaN
180,9.5,34,1,1,0,1,255,NaN
278,9.5,34,1,1,0,1,255,NaN
280,9.5,34,1,1,0,1,255,NaN
308,9.5,34,1,1,0,1,255,baixa


In [14]:
df['soma'] = df.iloc[:,2:6].sum(axis=1)
df

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade,soma
0,6.6,35,1,0,1,0,254,alta,2
1,6.6,36,0,1,0,1,253,alta,2
2,8.5,70,1,1,1,1,246,NaN,4
3,9.5,34,1,1,0,1,255,NaN,3
4,6.6,37,0,0,0,0,255,média,0
...,...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,NaN,2
1055,6.7,38,1,0,1,0,255,NaN,2
1056,3.0,40,1,1,1,1,255,NaN,4
1057,6.8,43,1,0,1,0,250,NaN,2


In [15]:
conditions = (df['pH'].between(6.5, 6.8)) & (df['Temperatura'].between(35.0, 45.0)) & (df['soma'] >= 3)
df.loc[df['Qualidade'].isnull() & conditions, 'Qualidade'] = 'alta'
df

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade,soma
0,6.6,35,1,0,1,0,254,alta,2
1,6.6,36,0,1,0,1,253,alta,2
2,8.5,70,1,1,1,1,246,NaN,4
3,9.5,34,1,1,0,1,255,NaN,3
4,6.6,37,0,0,0,0,255,média,0
...,...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,NaN,2
1055,6.7,38,1,0,1,0,255,NaN,2
1056,3.0,40,1,1,1,1,255,NaN,4
1057,6.8,43,1,0,1,0,250,NaN,2


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pH           1059 non-null   float64
 1   Temperatura  1059 non-null   int64  
 2   Sabor        1059 non-null   int64  
 3   Odor         1059 non-null   int64  
 4   Gordura      1059 non-null   int64  
 5   Turbidez     1059 non-null   int64  
 6   Cor          1059 non-null   int64  
 7   Qualidade    560 non-null    object 
 8   soma         1059 non-null   int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 74.6+ KB


In [17]:
conditions = (df['pH'].between(3.0, 9.5)) & (df['Temperatura'].between(35.0, 45.0)) & (df['soma'].between(1, 2))
df.loc[df['Qualidade'].isnull() & conditions, 'Qualidade'] = 'média'
df

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade,soma
0,6.6,35,1,0,1,0,254,alta,2
1,6.6,36,0,1,0,1,253,alta,2
2,8.5,70,1,1,1,1,246,NaN,4
3,9.5,34,1,1,0,1,255,NaN,3
4,6.6,37,0,0,0,0,255,média,0
...,...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,média,2
1055,6.7,38,1,0,1,0,255,média,2
1056,3.0,40,1,1,1,1,255,NaN,4
1057,6.8,43,1,0,1,0,250,média,2


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pH           1059 non-null   float64
 1   Temperatura  1059 non-null   int64  
 2   Sabor        1059 non-null   int64  
 3   Odor         1059 non-null   int64  
 4   Gordura      1059 non-null   int64  
 5   Turbidez     1059 non-null   int64  
 6   Cor          1059 non-null   int64  
 7   Qualidade    774 non-null    object 
 8   soma         1059 non-null   int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 74.6+ KB


In [19]:
df['Qualidade'].fillna('baixa', inplace=True)
df

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade,soma
0,6.6,35,1,0,1,0,254,alta,2
1,6.6,36,0,1,0,1,253,alta,2
2,8.5,70,1,1,1,1,246,baixa,4
3,9.5,34,1,1,0,1,255,baixa,3
4,6.6,37,0,0,0,0,255,média,0
...,...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,média,2
1055,6.7,38,1,0,1,0,255,média,2
1056,3.0,40,1,1,1,1,255,baixa,4
1057,6.8,43,1,0,1,0,250,média,2


In [20]:
df.drop('soma', axis=1, inplace=True)
df

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
2,8.5,70,1,1,1,1,246,baixa
3,9.5,34,1,1,0,1,255,baixa
4,6.6,37,0,0,0,0,255,média
...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,média
1055,6.7,38,1,0,1,0,255,média
1056,3.0,40,1,1,1,1,255,baixa
1057,6.8,43,1,0,1,0,250,média


In [21]:
X = df.drop('Qualidade', axis=1)
y = df['Qualidade']

In [22]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y

array([0, 0, 1, ..., 1, 2, 1])

In [23]:
scaler = StandardScaler()
X_norm = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_norm

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
0,-0.021531,-0.914107,0.910503,-0.872962,0.699608,-0.982217,0.501600
1,-0.021531,-0.815035,-1.098294,1.145525,-1.429372,1.018105,0.269333
2,1.336564,2.553438,0.910503,1.145525,0.699608,1.018105,-1.356536
3,2.051351,-1.013180,0.910503,1.145525,-1.429372,1.018105,0.733867
4,-0.021531,-0.715962,-1.098294,-0.872962,-1.429372,-0.982217,0.733867
...,...,...,...,...,...,...,...
1054,0.049947,0.076620,0.910503,1.145525,-1.429372,-0.982217,-1.124269
1055,0.049947,-0.616889,0.910503,-0.872962,0.699608,-0.982217,0.733867
1056,-2.594765,-0.418744,0.910503,1.145525,0.699608,1.018105,0.733867
1057,0.121426,-0.121525,0.910503,-0.872962,0.699608,-0.982217,-0.427468


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
np.unique(y_train_balanced, return_counts=True)

(array([0, 1, 2]), array([375, 375, 375]))

In [25]:
# label_encoder = LabelEncoder()
# y_train_balanced = label_encoder.fit_transform(y_train_balanced)
# y_train_balanced

In [26]:
# np.unique(y_train_balanced, return_counts=True)

In [27]:
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_balanced, y_train_balanced)

SVC(kernel='linear', random_state=42)

In [28]:
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        45
           1       0.89      0.72      0.80        94
           2       0.75      0.93      0.83        73

    accuracy                           0.80       212
   macro avg       0.80      0.80      0.79       212
weighted avg       0.81      0.80      0.80       212



### Unsupervisioned Model

In [29]:
dataset = pd.read_csv(path)
dataset

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
2,8.5,70,1,1,1,1,246,NaN
3,9.5,34,1,1,0,1,255,NaN
4,6.6,37,0,0,0,0,255,média
...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,NaN
1055,6.7,38,1,0,1,0,255,NaN
1056,3.0,40,1,1,1,1,255,NaN
1057,6.8,43,1,0,1,0,250,NaN


In [30]:
data = dataset
data

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
2,8.5,70,1,1,1,1,246,NaN
3,9.5,34,1,1,0,1,255,NaN
4,6.6,37,0,0,0,0,255,média
...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,NaN
1055,6.7,38,1,0,1,0,255,NaN
1056,3.0,40,1,1,1,1,255,NaN
1057,6.8,43,1,0,1,0,250,NaN


In [31]:
dataset_pred = data[data.Qualidade.isnull()].drop('Qualidade', axis=1)

In [32]:
dataset_pred

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
2,8.5,70,1,1,1,1,246
3,9.5,34,1,1,0,1,255
5,6.6,37,1,1,1,1,255
8,8.1,66,1,0,1,1,255
10,6.7,45,1,1,1,0,245
...,...,...,...,...,...,...,...
1053,8.1,66,1,0,1,1,255
1054,6.7,45,1,1,0,0,247
1055,6.7,38,1,0,1,0,255
1056,3.0,40,1,1,1,1,255


In [33]:
dataset_pred_norm = pd.DataFrame(scaler.transform(dataset_pred), columns = dataset_pred.columns)
dataset_pred_norm

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
0,1.336564,2.553438,0.910503,1.145525,0.699608,1.018105,-1.356536
1,2.051351,-1.013180,0.910503,1.145525,-1.429372,1.018105,0.733867
2,-0.021531,-0.715962,0.910503,1.145525,0.699608,1.018105,0.733867
3,1.050649,2.157147,0.910503,-0.872962,0.699608,1.018105,0.733867
4,0.049947,0.076620,0.910503,1.145525,0.699608,-0.982217,-1.588803
...,...,...,...,...,...,...,...
630,1.050649,2.157147,0.910503,-0.872962,0.699608,1.018105,0.733867
631,0.049947,0.076620,0.910503,1.145525,-1.429372,-0.982217,-1.124269
632,0.049947,-0.616889,0.910503,-0.872962,0.699608,-0.982217,0.733867
633,-2.594765,-0.418744,0.910503,1.145525,0.699608,1.018105,0.733867


In [34]:
y_pred2 = svm.predict(dataset_pred_norm)
y_pred2

array([1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 1, 2, 0, 0, 0, 2, 2, 2, 0, 1, 1, 0,
       2, 2, 0, 1, 1, 1, 1, 2, 1, 0, 2, 2, 1, 2, 0, 0, 2, 2, 1, 2, 2, 1,
       2, 2, 0, 2, 1, 1, 0, 1, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2,
       2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 1, 2, 0,
       1, 1, 2, 0, 0, 1, 1, 2, 0, 0, 0, 2, 2, 0, 0, 1, 2, 2, 1, 2, 1, 2,
       0, 1, 2, 2, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1,
       0, 1, 1, 2, 0, 2, 0, 2, 1, 1, 2, 2, 0, 2, 1, 1, 1, 1, 0, 2, 2, 1,
       2, 1, 2, 2, 0, 2, 1, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       2, 0, 1, 0, 1, 2, 0, 1, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 1, 2, 0, 1,
       2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2, 0,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 1, 0,
       2, 1, 2, 2, 1, 2, 0, 0, 1, 2, 0, 1, 2, 0, 1, 1, 2, 0, 1, 0, 1, 1,
       2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 1,
       1, 2, 2, 0, 2, 0, 0, 1, 2, 2, 1, 1, 0, 2, 2,

In [35]:
X_train_new = pd.concat([X_train, dataset_pred_norm], ignore_index=True)
X_train_new.shape

(1482, 7)

In [36]:
y_train_new = pd.concat([pd.Series(y_train), pd.Series(y_pred2)], ignore_index=True)
y_train_new.shape

(1482,)

In [37]:
ps_labeling = SVC(kernel='linear', random_state=42)
ps_labeling.fit(X_train_new, y_train_new)

SVC(kernel='linear', random_state=42)

In [38]:
y_pred3 = ps_labeling.predict(X_test)
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        45
           1       0.89      0.72      0.80        94
           2       0.75      0.93      0.83        73

    accuracy                           0.80       212
   macro avg       0.80      0.80      0.79       212
weighted avg       0.81      0.80      0.80       212



In [39]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        45
           1       0.89      0.72      0.80        94
           2       0.75      0.93      0.83        73

    accuracy                           0.80       212
   macro avg       0.80      0.80      0.79       212
weighted avg       0.81      0.80      0.80       212



### Self Train

In [40]:
data = pd.read_csv(path)
dataset_no_label = data[data['Qualidade'].isnull()]
data.dropna(inplace=True)
data

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
4,6.6,37,0,0,0,0,255,média
6,5.5,45,1,0,1,1,250,baixa
7,4.5,60,0,1,1,1,250,baixa
...,...,...,...,...,...,...,...,...
1047,6.8,45,1,1,1,0,245,alta
1048,9.5,34,1,1,0,1,255,baixa
1049,6.5,37,0,0,0,0,255,média
1052,6.5,40,1,0,0,0,250,média


In [41]:
data['Qualidade'] = label_encoder.fit_transform(data['Qualidade'])
data

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,0
1,6.6,36,0,1,0,1,253,0
4,6.6,37,0,0,0,0,255,2
6,5.5,45,1,0,1,1,250,1
7,4.5,60,0,1,1,1,250,1
...,...,...,...,...,...,...,...,...
1047,6.8,45,1,1,1,0,245,0
1048,9.5,34,1,1,0,1,255,1
1049,6.5,37,0,0,0,0,255,2
1052,6.5,40,1,0,0,0,250,2


In [42]:
X_ = data.drop('Qualidade', axis=1)
y_ = data['Qualidade']
scaler = StandardScaler()
X_norm_self = pd.DataFrame(scaler.fit_transform(X_), columns=X_.columns)
X_norm_self.shape, y_.shape

((424, 7), (424,))

In [43]:
X_train_self, X_test_self, y_train_self, y_test_self = train_test_split(X_norm_self, y_, test_size=0.15, random_state=42, stratify=y_)

In [44]:
X_train_self.shape, X_test_self.shape, y_train_self.shape, y_test_self.shape

((360, 7), (64, 7), (360,), (64,))

In [45]:
svm_self = SVC(kernel='linear', probability=True, random_state=42)
svm_self.fit(X_train_self, y_train_self)

SVC(kernel='linear', probability=True, random_state=42)

In [46]:
dataset_no_label.drop('Qualidade', axis=1, inplace=True)

In [47]:
dataset_no_label_norm = pd.DataFrame(scaler.fit_transform(dataset_no_label), columns=dataset_no_label.columns)
dataset_no_label_norm

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
0,1.368676,2.547679,0.908137,1.101156,0.690467,1.020686,-1.375222
1,2.092933,-1.033452,0.908137,1.101156,-1.448296,1.020686,0.729033
2,-0.007414,-0.735025,0.908137,1.101156,0.690467,1.020686,0.729033
3,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033
4,0.065012,0.060782,0.908137,1.101156,0.690467,-0.979733,-1.609028
...,...,...,...,...,...,...,...
630,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033
631,0.065012,0.060782,0.908137,1.101156,-1.448296,-0.979733,-1.141416
632,0.065012,-0.635549,0.908137,-0.908137,0.690467,-0.979733,0.729033
633,-2.614741,-0.436597,0.908137,1.101156,0.690467,1.020686,0.729033


In [48]:
prob = svm_self.predict_proba(dataset_no_label_norm)
y_pred_label = svm_self.predict(dataset_no_label_norm)

In [49]:
df_results = pd.DataFrame(prob, columns=['Prob c0', 'Prob c1', 'Prob c2'])
df_results['Predict'] = y_pred_label
df_results['Max_Prob'] = df_results[['Prob c0', 'Prob c1', 'Prob c2']].max(axis=1)

In [50]:
df_results

,Prob c0,Prob c1,Prob c2,Predict,Max_Prob
0,1.126039e-07,0.999999,8.554657e-07,1,0.999999
1,1.872721e-01,0.692435,1.202929e-01,1,0.692435
2,7.708412e-01,0.228844,3.145274e-04,0,0.770841
3,6.865699e-09,1.000000,1.516240e-07,1,1.000000
4,5.353607e-01,0.177515,2.871242e-01,0,0.535361
...,...,...,...,...,...
630,6.865699e-09,1.000000,1.516240e-07,1,1.000000
631,2.544382e-02,0.105825,8.687316e-01,2,0.868732
632,5.120100e-01,0.254953,2.330369e-01,0,0.512010
633,1.543550e-01,0.836735,8.910009e-03,1,0.836735


In [51]:
dataset_no_label_norm[['predict', 'max_prob']] = df_results[['Predict','Max_Prob']]
dataset_no_label_norm

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,predict,max_prob
0,1.368676,2.547679,0.908137,1.101156,0.690467,1.020686,-1.375222,1,0.999999
1,2.092933,-1.033452,0.908137,1.101156,-1.448296,1.020686,0.729033,1,0.692435
2,-0.007414,-0.735025,0.908137,1.101156,0.690467,1.020686,0.729033,0,0.770841
3,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033,1,1.000000
4,0.065012,0.060782,0.908137,1.101156,0.690467,-0.979733,-1.609028,0,0.535361
...,...,...,...,...,...,...,...,...,...
630,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033,1,1.000000
631,0.065012,0.060782,0.908137,1.101156,-1.448296,-0.979733,-1.141416,2,0.868732
632,0.065012,-0.635549,0.908137,-0.908137,0.690467,-0.979733,0.729033,0,0.512010
633,-2.614741,-0.436597,0.908137,1.101156,0.690467,1.020686,0.729033,1,0.836735


In [52]:
dataset_no_label_final = dataset_no_label_norm.loc[dataset_no_label_norm.max_prob >= 0.75]
dataset_no_label_final

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,predict,max_prob
0,1.368676,2.547679,0.908137,1.101156,0.690467,1.020686,-1.375222,1,0.999999
2,-0.007414,-0.735025,0.908137,1.101156,0.690467,1.020686,0.729033,0,0.770841
3,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033,1,1.000000
5,-0.731671,0.558161,-1.101156,1.101156,0.690467,1.020686,0.729033,1,0.833121
6,1.441102,1.055541,-1.101156,1.101156,0.690467,1.020686,0.729033,1,0.835010
...,...,...,...,...,...,...,...,...,...
628,-0.007414,-0.735025,0.908137,1.101156,0.690467,1.020686,0.729033,0,0.770841
629,-0.804097,0.060782,0.908137,-0.908137,0.690467,1.020686,-0.439997,1,0.973751
630,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033,1,1.000000
631,0.065012,0.060782,0.908137,1.101156,-1.448296,-0.979733,-1.141416,2,0.868732


In [53]:
data.dropna(inplace=True)
X_data = data.drop('Qualidade', axis=1)
y_data = data['Qualidade']
scaler = StandardScaler()
X_norm_data = pd.DataFrame(scaler.fit_transform(X_data), columns=X_data.columns)
X_norm_data.shape, y_data.shape

((424, 7), (424,))

In [54]:
X_concat = pd.concat([X_norm_data, dataset_no_label_final.iloc[:,0:7]], ignore_index=True)
X_concat

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
0,-0.042026,-0.885100,0.914056,-0.822125,0.713370,-0.985948,0.511117
1,-0.042026,-0.786575,-1.094025,1.216360,-1.401797,1.014253,0.281060
2,-0.042026,-0.688050,-1.094025,-0.822125,-1.401797,-0.985948,0.741174
3,-0.813707,0.100152,0.914056,-0.822125,0.713370,1.014253,-0.409111
4,-1.515236,1.578029,-1.094025,1.216360,0.713370,1.014253,-0.409111
...,...,...,...,...,...,...,...
778,-0.007414,-0.735025,0.908137,1.101156,0.690467,1.020686,0.729033
779,-0.804097,0.060782,0.908137,-0.908137,0.690467,1.020686,-0.439997
780,1.078973,2.149775,0.908137,-0.908137,0.690467,1.020686,0.729033
781,0.065012,0.060782,0.908137,1.101156,-1.448296,-0.979733,-1.141416


In [55]:
data

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,0
1,6.6,36,0,1,0,1,253,0
4,6.6,37,0,0,0,0,255,2
6,5.5,45,1,0,1,1,250,1
7,4.5,60,0,1,1,1,250,1
...,...,...,...,...,...,...,...,...
1047,6.8,45,1,1,1,0,245,0
1048,9.5,34,1,1,0,1,255,1
1049,6.5,37,0,0,0,0,255,2
1052,6.5,40,1,0,0,0,250,2


In [56]:
y_concat = pd.concat([data['Qualidade'], dataset_no_label_final.iloc[:,7]], ignore_index=True)
y_concat

0      0
1      0
2      2
3      1
4      1
      ..
778    0
779    1
780    1
781    2
782    1
Length: 783, dtype: int64

In [57]:
X_train_concat, X_test_concat, y_train_concat, y_test_concat = \
    train_test_split(X_concat, y_concat, test_size=0.15, random_state=42, stratify=y_concat)

In [58]:
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear', 'poly', 'rbf']}

In [59]:
svm_concat = SVC()
grid = GridSearchCV(svm_concat, param_grid, refit=True, verbose=3, cv=5)
grid.fit(X_train_concat, y_train_concat)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.857 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.865 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.812 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.850 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.910 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.962 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.962 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.962 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.962 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.955 total time=   0.0s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.940 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'poly', 'rbf']},
             verbose=3)

In [60]:
print(grid.best_params_)

{'C': 100, 'gamma': 1, 'kernel': 'rbf'}


In [61]:
svm_concat_CV = SVC(kernel='linear', probability=True, random_state=42)
svm_concat_CV.fit(X_train_concat, y_train_concat)

SVC(kernel='linear', probability=True, random_state=42)

In [62]:
y_pred_concat = svm_concat_CV.predict(X_test_concat)

In [63]:
print(f'1º Modelo:\n{classification_report(y_test, y_pred)}')
print(f'2º Modelo:\n{classification_report(y_test, y_pred3)}')
print(f'3º Modelo:\n{classification_report(y_test_concat, y_pred_concat)}')

1º Modelo:
              precision    recall  f1-score   support

           0       0.76      0.76      0.76        45
           1       0.89      0.72      0.80        94
           2       0.75      0.93      0.83        73

    accuracy                           0.80       212
   macro avg       0.80      0.80      0.79       212
weighted avg       0.81      0.80      0.80       212

2º Modelo:
              precision    recall  f1-score   support

           0       0.76      0.76      0.76        45
           1       0.89      0.72      0.80        94
           2       0.75      0.93      0.83        73

    accuracy                           0.80       212
   macro avg       0.80      0.80      0.79       212
weighted avg       0.81      0.80      0.80       212

3º Modelo:
              precision    recall  f1-score   support

           0       0.74      0.94      0.83        18
           1       0.98      0.94      0.96        51
           2       1.00      0.94      0.9

In [64]:
with open('std_scaler', 'wb') as scaler_file:
    pk.dump(scaler, scaler_file)

In [65]:
with open('label_model', 'wb') as model_file:
    pk.dump(svm_concat_CV, model_file)

In [66]:
model_scaler = pd.read_pickle('/content/std_scaler')
model_label = pd.read_pickle('/content/label_model')

In [67]:
new_data = {
    'pH':[6.4],
    'Temperatura':[52],
    'Sabor':[0],
    'Odor':[1],
    'Gordura':[1],
    'Turbidez':[0],
    'Cor':[253]
}

In [68]:
new_data = pd.DataFrame(new_data)
new_data

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor
0,6.4,52,0,1,1,0,253


In [69]:
new_data = model_scaler.transform(new_data)
result = model_label.predict(new_data)
label_encoder.inverse_transform(result)

array(['alta'], dtype=object)